### Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import duckdb
from edgar import Company, find, set_identity, MultiFinancials
set_identity("torbj_storli@gmail.com")

### Get data from EDGAR using edgartools package

In [ ]:
company = Company("ABBV")
company

### Create a XBRL object that holds ds the latest 10-Q filing

In [ ]:
filing = company.get_filings(form="10-Q").latest()

In [ ]:
text = filing.text()  # Clean, readable text
print(text)

### Get the company's latest 10K Filing

In [ ]:
filing = company.get_filings(form="10-K").latest()

In [ ]:
text = filing.text()  # Clean, readable text
print(text)

### Create multi-period financials

In [ ]:
filings = company.get_filings(form="10-K").head(6)

In [ ]:
multi_financials = MultiFinancials.extract(filings)

# Access statements spanning multiple years
balance_sheet = multi_financials.balance_sheet()
income_statement = multi_financials.income_statement()
cash_flow = multi_financials.cashflow_statement()

In [ ]:
# Query specific financial items
revenue = income_statement.statement_data.get("Revenue")
net_income = income_statement.statement_data.get("Net Income")
total_assets = balance_sheet.statement_data.get("Total Assets")

In [ ]:
print(revenue)
print(net_income)
print(total_assets)

### Print object Type

In [ ]:
print(type(multi_financials))
print(type(income_statement))

### Print Multi-Year Income Statement

In [ ]:
print("Multi-Year Income Statement")
print(income_statement)

### Print Multi-Year Balance Sheet

In [ ]:
print("Multi-Year Balance Sheet")
print(balance_sheet)

### Print Multi-Year Cash Flow Statement

In [ ]:
print("Multi-Year Cash Flow Statement")
print(cash_flow)

### Create a list of the Income Statement's Column Names

In [ ]:
is_df = income_statement.to_dataframe() 
is_all_cols = is_df.columns.tolist()
is_all_cols

### Reduce dataframe to only include columns we need

In [ ]:
is_all_cols = ['label',
 'concept',
 '2024-12-31',
 '2023-12-31',
 '2022-12-31',
 '2021-12-31',
 '2020-12-31',
 '2019-12-31'
]

In [ ]:
is_cols = [x for x in is_all_cols if x != 'concept']
is_cols


In [ ]:
#is_cols = is_all_cols  
#is_cols.pop(1) # Removes 'concept' at index 1
#is_df = income_statement.to_dataframe() 
is_df = is_df[is_cols]
is_df

### Create a Balance Sheet Dataframe

In [ ]:
bs_df = balance_sheet.to_dataframe()
bs_df

### Create a list of the Balance Sheet's Column Names

In [ ]:
bs_all_cols = bs_df.columns.tolist()
bs_all_cols

### Reduce dataframe to only include columns we need

In [ ]:
bs_cols = bs_all_cols  
bs_cols.pop(1) # Removes 'concept' at index 1
bs_df = bs_df[bs_cols]
bs_df

### Create a Cash Flow Statement Dataframe

In [ ]:
cf_df = cash_flow.to_dataframe()
cf_df 

### Create a list of the Cash Flow Statemenment's Column Names

In [ ]:
cf_all_cols = cf_df.columns.tolist()
cf_all_cols

### Reduce dataframe to only include columns we need

In [ ]:
cf_cols = cf_all_cols  
cf_cols.pop(1) # Removes 'concept' at index 1
cf_df = cf_df[cf_cols]
cf_df

### Create a new DuckDB Database

In [ ]:
#conn.close()
conn = duckdb.connect("financials.db")

### Create tables to hold the
- Income Statement
- Balance Sheet
- Cash Flow statement data

In [ ]:
def optimize_financial_df(df, cols):
    """Optimize data types for financial dataframes and select only needed columns"""
    optimized_df = df[cols].copy()
    
    # Get the number of columns to determine how many numeric columns to convert
    num_cols = len(optimized_df.columns)
    
    # Convert all columns from position 1 onwards to numeric (skipping 'label')
    for i in range(1, num_cols):
        optimized_df.iloc[:, i] = pd.to_numeric(optimized_df.iloc[:, i], errors='coerce')
        # Note: errors='coerce' -> If a value cannot be converted to a number → replaces 
        # it with NaN (Not a Number) and continues processing without throwing an error 
        
    # Create dtype dictionary for all numeric columns
    dtype_dict = {}
    for i in range(1, num_cols):
        dtype_dict[optimized_df.columns[i]] = 'float64'
    
    # Apply the dtype changes
    optimized_df = optimized_df.astype(dtype_dict)
    
    return optimized_df

### Optimize the dataframes

In [ ]:
income_statement_opt = optimize_financial_df(is_df, is_cols)
balance_sheet_opt = optimize_financial_df(bs_df, bs_cols)
cash_flow_opt = optimize_financial_df(cf_df, cf_cols)

### Verify that dataframe data is valid

In [ ]:
print(income_statement_opt.info())
print(balance_sheet_opt.info())
print(cash_flow_opt.info())

### Clean the dataframes

In [ ]:
def clean_financial_dataframe(df):
    """Clean financial DataFrame by converting all non-label columns to numeric"""
    df_clean = df.copy()
    
    for col in df_clean.columns:
        if col != 'label':
            # Replace empty strings, 'None', 'NaN' strings with actual NaN
            df_clean[col] = df_clean[col].replace(['', 'None', 'NaN', None], np.nan)
            # Convert to numeric, coercing errors to NaN
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    return df_clean

In [ ]:
# Clean DataFrames
income_statement_opt_clean = clean_financial_dataframe(income_statement_opt)
balance_sheet_opt_clean = clean_financial_dataframe(balance_sheet_opt)
cash_flow_opt_clean = clean_financial_dataframe(cash_flow_opt)

# Check the result
print("After cleaning:")
print(balance_sheet_opt_clean.info())
print(balance_sheet_opt_clean.dtypes)


### Create Tables in DuckDB

In [ ]:
def build_create_table_query(cols, table_name, source_table):
    """
    Build a dynamic CREATE TABLE AS SELECT query for DuckDB.
    
    Args:
        cols (list): List of column names
        table_name (str): Name of the table to create
        source_table (str): Name of the source table to select from
    
    Returns:
        str: The complete SQL query string
    """
    select_clauses = [
        f'{col}::VARCHAR as {col}' if col == 'label' 
        else f'"{col}"::DOUBLE as "{col}"'
        for col in cols
    ]
    
    query = f"""CREATE TABLE {table_name} AS 
SELECT 
    {',\n    '.join(select_clauses)}
FROM {source_table}"""
    
    return query


    # Use with conn.execute:
    # conn.execute(query)

### See if any Tables Exists

In [ ]:
conn.sql("SHOW Tables")

### Create and Execute DuckDB Insert Statements

In [ ]:
queryIS = build_create_table_query(is_cols, 'income_statement', 'income_statement_opt_clean')
print(queryIS)
conn.execute(queryIS)

queryBS = build_create_table_query(bs_cols, 'balance_sheet', 'balance_sheet_opt_clean')
print(queryBS)
conn.execute(queryBS)

queryCF = build_create_table_query(cf_cols, 'cash_flow', 'cash_flow_opt_clean')
print(queryCF)    
conn.execute(queryCF)

### Create DataFrames for each table

In [ ]:
income_sample = conn.execute("SELECT * FROM income_statement LIMIT 5").df()
balance_sample = conn.execute("SELECT * FROM balance_sheet LIMIT 5").df()
cash_flow_sample = conn.execute("SELECT * FROM cash_flow LIMIT 5").df()

In [ ]:
income_sample

In [ ]:
balance_sample

In [ ]:
cash_flow_sample

#### Close Connection to DuckDB

In [ ]:
conn.close()

### Show all EdgarTools available attributes/classes

In [ ]:
import edgar
print(dir(edgar))  # Shows all available attributes/classes

# Or use help
#help(edgar)


#### <center>END OF SCRIPT</center> 